# **Hugging Face NLP Course - Chapter 4**

Source: https://huggingface.co/learn/nlp-course/chapter4/1

---

In [ ]:
!pip install datasets evaluate transformers[sentencepiece]

## **4. Sharing Models And Tokenizers**

### **4.1. The HuggingFace Hub**

The [Hugging Face Hub](https://huggingface.co/) is a central platform that enables anyone to discover, use, and contribute new state-of-the-art models and datasets. We'll focus on the models in this chapter, and take a look at the datasets in chapter 5.

The models in the Hub are not limited to HF Transformers or even NLP. There are models from [Flair](https://github.com/flairNLP/flair) and [AllenNLP](https://github.com/allenai/allennlp) for NLP, [Asteroid](https://github.com/asteroid-team/asteroid) and [pyannote](https://github.com/pyannote/pyannote-audio) for speech, and [timm](https://github.com/rwightman/pytorch-image-models) for vision, to name a few.

Each of these models is hosted as a Git repository, which allows versioning and reproducibility. Sharing a model on the Hub means opening it up to the community and making it accessible to anyone looking to easily use it, in turn eliminating their need to train a model on their own and simplifying sharing and usage.

Additionally, sharing a model on the Hub automatically deploys a hosted Inference API for that model. Anyone in the community is free to test it out directly on the model's page, with custom inputs and appropriate widgets.



### **4.2. Using pretrained models**

The Model Hub makes selecting the appropriate model simple, so that using it in any downstream library can be done in a few lines of code. Let's take a look at how to actually use one of these models, and how to contribute back to the community.

Let's say we're looking for a French-based model that can perform mask filling. We select the `camembert-base` checkpoint to try it out. The identifier `camembert-base` is all we need to start using it. As you've seen in previous chapters, we can instantiate it using the `pipeline()` function.

In [ ]:
from transformers import pipeline

camembert_fill_mask = pipeline("fill-mask", model="camembert-base")
results = camembert_fill_mask("Le camembert est <mask> :)")

As you can see, loading a model within a pipeline is extremely simple. The only thing you need to watch out for is that the chosen checkpoint is suitable for the task it's going to be used for. For example, here we are loading the `camembert-base` checkpoint in the `fill-mask` pipeline, which is completely fine. But if we were to load this checkpoint in the `text-classification` pipeline, the results would not make any sense because the head of `camembert-base` is not suitable for this task. We recommend using the task selector in the Hugging Face Hub interface in order to select the appropriate checkpoints. You can also instantiate the checkpoint using the model architecture directly:

In [ ]:
from transformers import CamembertTokenizer, CamembertForMaskedLM
# TensorFlow: from transformers import CamembertTokenizer, TFCamembertForMaskedLM

tokenizer = CamembertTokenizer.from_pretrained("camembert-base")
model = CamembertForMaskedLM.from_pretrained("camembert-base")

However, we recommend using the [`Auto*` classes](https://huggingface.co/transformers/model_doc/auto.html?highlight=auto#auto-classes) instead, as these are by design architecture-agnostic. While the previous code sample limits users to checkpoints loadable in the CamemBERT architecture, using the `Auto*` classes makes switching checkpoints simple:

In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
# TensorFlow: from transformers import AutoTokenizer, TFAutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("camembert-base")
model = AutoModelForMaskedLM.from_pretrained("camembert-base")

Hint: When using a pretrained model, make sure to check how it was trained, on which datasets, its limits, and its biases. All of this information should be indicated on its model card.

### **4.3. Sharing pretrained models**

In the steps below, we'll take a look at the easiest ways to share pretrained odels to the HF Hub.

There are three ways to go about creating new model repositories:
- Using the `push_to_hub` API
- Using the `huggingface_hub` Python library
- Using the web interface

Once you've created a repository, you can upload files to it via git and git-lfs.

#### **Using the `push_to_hub` API**

The simplest way to upload files to the Hub is by leveraging the `push_to_hub` API.

Before going further, you'll need to generate an authentication token so that the `huggingface_hub` API knows who you are and what namespaces you have write access to. Make sure you are in an environment where you have `transformers` installed. If you are in a notebook, you can use the following function to login:

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In a terminal, you can run:

In [ ]:
!huggingface-cli login

In both cases, you should be prompted for your username and password, which are the same ones you use to log in to the Hub.

Great! You now have your authentication token stored in your cache folder. Let's create some repositories!

##### **Upload with PyTorch**

If you have played around with the `Trainer` API to train a model, the easiest way to upload it to the Hub is to set `push_to_hub=True` when you define your `TrainingArguments`:

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    "bert-finetuned-mrpc",
    save_strategy="epoch",
    push_to_hub=True,
)

When you call `trainer.train()`, the `Trainer` will then upload your model to the Hub each time it is saved (here every epoch) in a repository in your namespace. That repository will be named like the output directory you picked (here `bert-finetuned-mrpc`) but you can choose a different name with `hub_model_id="a_different_name"`.

To upload your model to an organization you are a member of, just pass it with `hub_model_id="my_organization/my_repo_name"`.

Once your training is finished, you should do a final `trainer.push_to_hub()` to upload the last version of your model. It will also generate a model card with all the relevant metadata, reporting the hyperparameters used and the evaluation results (e.g. training loss, validation loss, accuracy, F1 score, combined score).

##### **Upload with TensorFlow**

If you are using Keras to train your model, the easiest way to upload it to the Hub is to pass along a `PushToHubCallback` when you call `model.fit()`:

In [ ]:
from transformers import PushToHubCallback

callback = PushToHubCallback(
    "bert-finetuned-mrpc",
    save_strategy="epoch",
    tokenizer=tokenizer,
)

Then you should add `callbacks=[callback]` in your call to `model.fit()`. The callback will then upload your model to the Hub each time it is saved (here every epoch) in a repository in your namespace. That repository will be named like the output directory you picked (here `bert-finetuned-mrpc`) but you can choose a different name with `hub_model_id="a_different_name"`.

To upload your model to an organization you are a member of, just pass it with `hub_model_id="my_organization/my_repo_name"`.

##### **Continue with either PyTorch or TensorFlow**

At a lower level, accessing the Model Hub can be done directly on models, tokenizers, and configuration objects via their `push_to_hub()` method. This method takes care of both the repository creation and pushing the model and tokenizer files directly to the repository. No manual handling is required, unlike with the API we'll see below.

To get an idea of how it works, let's first initialize a model and a tokenizer:

In [ ]:
from transformers import AutoModelForMaskedLM, AutoTokenizer
# TensorFlow: from transformers import TFAutoModelForMaskedLM, AutoTokenizer

checkpoint = "camembert-base"

model = AutoModelForMaskedLM.from_pretrained(checkpoint)
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

You're free to do whatever you want with these - add tokens to the tokenizer, train the model, fine-tune it. Once you're happy with the resulting model, weights, and tokenizer, you can leverage the `push_to_hub()` method directly available on the `model` object:

In [ ]:
model.push_to_hub("dummy-model")

This will create the new repository `dummy-model` in your profile, and populate it with your model files. Do the same with the tokenizer, so that all the files are now available in this repository:

In [ ]:
tokenizer.push_to_hub(
    "dummy-model",
    organization="huggingface",  # Specify this to upload to an organization's namespace
    use_auth_token="<TOKEN>",    # Specify this to use a specific Hugging Face token
)

As you've seen, the `push_to_hub()` method accepts several arguments, making it possible to upload to a specific repository or organization namespace, or to use a different API token. We recommend you take a look at the method specification available directly in the [HF Transformers documentation](https://huggingface.co/transformers/model_sharing.html) to get an idea of what is possible.

The `push_to_hub()` method is backed by the [`huggingface_hub`](https://github.com/huggingface/huggingface_hub) Python package, which offers a direct API to the Hugging Face Hub. It's integrated within HF Transformers and several other machine learning libraries, like [`allennlp`](https://github.com/allenai/allennlp). Although we focus on the HF Transformers integration in this chapter, integrating it into your own code or library is simple.

Jump to the last section to see how to upload files to your newly created repository!

#### **Using the `huggingface_hub` Python library**

The `huggingface_hub` Python library is a package which offers a set of tools for the model and datasets hubs. It provides simple methods and classes for common tasks like getting information about repositories on the hub and managing them. It provides simple APIs that work on top of git to manage those repositories' content and to integrate the Hub in your projects and libraries.

Similarly to using the `push_to_hub` API, this will require you to have your API token saved in your cache. Use the `login` command from the CLI, as mentioned in the previous section.

The `huggingface_hub` package offers several methods and classes which are useful for our purpose. Firstly, there are a few methods to manage repository creation, deletion, and others:

In [ ]:
from huggingface_hub import (
    # User management
    login,
    logout,
    whoami,

    # Repository creation and management
    create_repo,
    delete_repo,
    update_repo_visibility,

    # Some methods to retrieve/change information about the content
    list_models,
    list_datasets,
    list_metrics,
    list_repo_files,
    upload_file,
    delete_file,
)

Additionally, it offers the very powerful `Repository` class to manage a local repository. We will explore these methods and that class in the next few section to understand how to leverage them.

The `create_repo` method can be used to create a new repository on the hub:

In [ ]:
from huggingface_hub import create_repo

# Create repository `dummy-model` in your namespace
create_repo("dummy-model")

# Create repo in an organization's namespace, assuming you belong to that organization
create_repo("dummy-model", organization="huggingface")

Other arguments for `create_repo` which may be useful are:
- `private`, in order to specify if the repository should be visible from others or not.
- `token`, if you would like to override the token stored in your cache by a given token.
- `repo_type`, if you would like to create a `dataset` or a `space` instead of a model. Accepted values are `"dataset"` and `"space"`.

Once the repository is created, we should add files to it! Jump to the next section to see the three ways this can be handled.

#### **Using the web interface**

The web interface offers tools to manage repositories directly in the Hub. Using the interface, you can easily create repositories, add files (even large ones!), explore models, visualize diffs, and much more.

To create a new repository, visit [huggingface.co/new](https://huggingface.co/new). Specify the owner of the repository, enter your model's name, and specify whether you want your model to be public or private.

Done! We'll take a look at how to add some new files next.

#### **Uploading the model files**

The system to manage files on the Hugging Face Hub is based on git for regular files, and git-lfs (which stands for [Git Large File Storage](https://git-lfs.github.com/)) for larger files.

In the next section, we go over three different ways of uploading files to the Hub: through `huggingface_hub` and through git commands.

##### **The `upload_file` approach**

Using `upload_file` does not require git and git-lfs to be installed on your system. It pushes files directly to the HF Hub using HTTP POST requests. A limitation of this approach is that it doesn't handle files that are larger than 5GB in size. If your files are larger than 5GB, please follow the two other methods detailed below.

The API may be used as follows:

In [ ]:
from huggingface_hub import upload_file

upload_file(
    "<path_to_file>/config.json",
    path_in_repo="config.json",
    repo_id="<namespace>/dummy-model",
)

This will upload the file `config.json` available at `<path_to_file>` to the root of the repository as `config.json`, to the `dummy-model` repository. Other arguments which may be useful are:
- `token`, if you would like to override the token stored in your cache by a given token.
- `repo_type`, if you would like to upload to a `dataset` or a `space` instead of a model. Accepted values are `"dataset"` and `"space"`.

##### **The `Repository` class**

The `Repository` class manages a local repository in a git-like manner. It abstracts most of the pain points one may have with git to provide all features that we require.

Using this class requires having git and git-lfs installed, so make sure you have [git-lfs](https://git-lfs.github.com/) installed and set up before you begin.

In order to start playing around with the repository we have just created, we can start by initialising it into a local folder by cloning the remote repository:

In [ ]:
from huggingface_hub import Repository

repo = Repository("<path_to_dummy_folder>", clone_from="<namespace>/dummy-model")

This created the folder `<path_to_dummy_folder>` in our working directory, containing all the files of the cloned repository.

From this point on, we may leverage several of the traditional git methods:

In [ ]:
repo.git_pull()
repo.git_add()
repo.git_commit()
repo.git_push()
repo.git_tag()
# ... and others!

We recommend taking a look at the `Repository` documentation available [here](https://github.com/huggingface/huggingface_hub/tree/main/src/huggingface_hub#advanced-programmatic-repository-management) for an overview of all available methods.

At present, we have a model and a tokenizer that we would like to push to the hub. We have successfully cloned the repository, we can therefore save the files within that repository.

We first make sure that our local clone is up to date by pulling the latest changes. Once that is done, we save the model and tokenizer files.

In [ ]:
# Pull latest changes
repo.git_pull()

# Save files
model.save_pretrained("<path_to_dummy_folder>")
tokenizer.save_pretrained("<path_to_dummy_folder>")

The folder at `<path_to_dummy_folder>` now contains all the model and tokenizer files. We follow the usual git workflow by adding files to the staging area, committing them and pushing them to the hub:

In [ ]:
repo.git_add()
repo.git_commit("Add model and tokenizer files")
repo.git_push()

Congratulations! You just pushed your first files on the hub.

##### **The git-based approach**

This is the very barebones approach to uploading files: we'll do so with git and git-lfs directly. Most of the difficulty is abstracted away by previous approaches, but there are a few caveats with the following method so we'll follow a more complex use-case.

Using this class requires having git and git-lfs installed, so make sure you have [git-lfs](https://git-lfs.github.com/) installed and set up before you begin.

First start by initializing git-lfs:

In [ ]:
!git lfs install

Once that's done, the first step is to clone your model repository:

In [ ]:
!git clone https://huggingface.co/<namespace>/<your-model-id>

With e.g. `dummy` as your model ID, you now have a folder named `dummy` in your working directory. You can `cd` into the folder and have a look at the contents:

In [ ]:
!cd dummy && ls

Adding a regular-sized file, such as a configuration file, a vocabulary file, or basically any file under a few megabytes, is done exactly as one would do it in any git-based system. However, bigger files must be registered through git-lfs in order to push them to _huggingface.co_.

Let's go back to Python for a bit to generate a model and tokenizer that we'd like to commit to our dummy repository:

In [ ]:
from transformers import AutoModelForMaskedLM, AutoTokenizer
# TensorFlow: from transformers import TFAutoModelForMaskedLM, AutoTokenizer

checkpoint = "camembert-base"

model = AutoModelForMaskedLM.from_pretrained(checkpoint)
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

# Do whatever with the model, train it, fine-tune it...

model.save_pretrained("<path_to_dummy_folder>")
tokenizer.save_pretrained("<path_to_dummy_folder>")

Now that we've saved some model and tokenizer artifacts, let's take another look at the `dummy` folder. If you look at the file sizes (for example, with `ls -lh`), you should see that the model state dict file (_pytorch_model.bin_) is the only outlier, at more than 400 MB.

Hint: When creating the repository from the web interface, the `.gitattributes` file is automatically set up to consider files with certain extensions, such as `.bin` and `.h5`, as large files, and git-lfs will track them with no necessary setup on your side.

We can now go ahead and proceed like we would usually do with traditional Git repositories. We can add all the files to Git's staging environment using the `git add` command:

In [ ]:
!git add .

We can then have a look at the files that are currently staged:

In [ ]:
!git status

Similarly, we can make sure that git-lfs is tracking the correct files by using its `status` command:

In [ ]:
!git lfs status

We can see that all files have either `Git` (for files with sizes smaller than the threshold) or `LFS` (for files with larger sizes) as a handler. Great!

Let's proceed to the final steps, committing and pushing to the _huggingface.co_ remote repository:

In [ ]:
!git commit -m "First model version"

Pushing can take a bit of time, depending on the speed of your internet connection and the size of your files:

In [ ]:
!git push

If we take a look at the model repository when this is finished, we can see all the recently added files.

### **4.4. Building a model card**

The model card is a file which is arguably as important as the model and tokenizer files in a model repository. It is the central definition of the model, ensuring reusability by fellow community members and reproducibility of results, and providing a platform on which other members may build their artifacts.

Documenting the training and evaluation process helps others understand what to expect of a model - and providing sufficient information regarding the data that was used and the preprocessing and postprocessing that were done ensures that the limitations, biases, and contexts in which the model is and is not useful can be identified and understood.

Therefore, creating a model card that clearly defines your model is a very important step. Here, we provide some tips that will help you with this. Creating the model card is done through the _README.md_ file, which is a Markdown file.

The model card usually starts with a very brief, high-level overview of what the model is for, followed by additional details in the following sections:
- Model description
- Intended uses & limitations
- How to use
- Limitations and bias
- Training data
- Training procedure
- Evaluation results

Let's take a look at what each of these sections should contain.

##### **Model description**

The model description provides basic details about the model. This includes the architecture, version, if it was introduced in a paper, if an original implementation is available, the author, and general information about the model. Any copyright should be attributed here. General information about training procedures, parameters, and important disclaimers can also be mentioned in this section.

##### **Intended uses & limitations**

Here you describe the use cases the model is intended for, including the languages, fields, and domains where it can be applied. This section of the model card can also document areas that are known to be out of scope for the model, or where it is likely to perform suboptimally.

##### **How to use**

This section should include some examples of how to use the model. This can showcase usage of the `pipeline()` function, usage of the model and tokenizer classes, and any other code you think might be helpful.

##### **Limitations and bias**

This section should be used to indicate that the model can/does have limitations and bias (and state which ones if possible). Add a code example that shows some bias if possible. Indicate that the bias will also affect all fine-tuned versions of the model.

##### **Training data**

This part should indicate which dataset(s) the model was trained on. A brief description of the dataset(s) is also welcome.

##### **Training procedure**

In this section you should describe all the relevant aspects of training that are useful from a reproducibility perspective. This includes any preprocessing and postprocessing that were done on the data, as well as details such as the number of epochs the model was trained for, the batch size, the learning rate, and so on.

##### **Variable and metrics**

Here you should describe the metrics you use for evaluation, and the different factors you are measuring. Mentioning which metric(s) were used, on which dataset and which dataset split, makes it easy to compare your model's performance to that of other models. These should be informed by the previous sections, such as the intended users and use cases.

##### **Evaluation results**

Finally, provide an indication of how well the model performs on the evaluation dataset. If the model uses a decision threshold, either provide the decision threshold used in the evaluation, or provide details on evaluation at different thresholds for the intended uses.

#### **Note**

Model cards are not a requirement when publishing models, and you don't need to include all of the sections described above when you make one. However, explicit documentation of the model can only benefit future users, so we recommend that you fill in as many of the sections as possible to the best of your knowledge and ability.

#### **Model card metadata**

If you have done a little exploring of the Hugging Face Hub, you should have seen that some models belong to certain categories: you can filter them by tasks, languages, libraries, and more. The categories a model belongs to are identified according to the metadata you add in the model card header.

For example, if you take a look at the [`camembert-base` model card](https://huggingface.co/camembert-base/blob/main/README.md), you should see the following lines in the model card header:

In [ ]:
---
language: fr
license: mit
datasets:
- oscar
---

This metadata is parsed by the Hugging Face Hub, which then identifies this model as being a French model, with an MIT license, trained on the Oscar dataset.

The [full model card specification](https://github.com/huggingface/hub-docs/blame/main/modelcard.md) allows specifying languages, licenses, tags, datasets, metrics, as well as the evaluation results the model obtained when training.